In [175]:
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import requests

## Store finder (HTML)

In [329]:
url = 'https://www.choicethediscountstore.com.au/store-finder/'
page = requests.get(url)
html = page.text
soup = BeautifulSoup(html, "html.parser")

In [407]:
store_info_dict = {'Store Name' : [],
                   'Latitude' : [],
                   'Longitude' : [],
                   'Store Address' : [],
                   'Suburb/City' : [],
                   'Postal Code' : [],
                   'Country' : [],
                   'Phone' : []}

store_details = {'Store Name' : [],
                 'Open Hours' : []
                 }

store_list = soup.find_all('div', class_ = 'card mb-2')

for store in store_list:
    
    ## Store Name / Brand Name extraction
    store_info_dict['Store Name'].append(store.find('a').get_text())
    
    ## Phone Number
    store_info_dict['Phone'].append(store.find('p').get_text(strip = True))
    
    
    ## Store address extraction
    add = store.find_all('p')[1]
    add_text = add.get_text(separator = '\n', strip = True).split('\n')
    
    if len(add_text) > 1:
        store_info_dict['Store Address'].append(add_text[0])
        add_text_splitted = add_text[1].split(', ')
        store_info_dict['Suburb/City'].append(add_text_splitted[0])
        store_info_dict['Postal Code'].append(add_text_splitted[1])
        store_info_dict['Country'].append(add_text_splitted[2])
    else:
        store_info_dict['Store Address'].append('')
        store_info_dict['Suburb/City'].append('')
        store_info_dict['Postal Code'].append('')
        store_info_dict['Country'].append('AU')
    
    ## Lat-lon
    map_link = store.find_all('a')
    for link in map_link:
        lat_lon = re.search('(?<=\/@)([-]?\d*\.\d+)\,([-]?\d*\.\d+)', str(link))
        if lat_lon is not None:
            break
    if lat_lon is not None:
        lat, lon = lat_lon[0].split(',')
    store_info_dict['Latitude'].append(lat)
    store_info_dict['Longitude'].append(lon)
    
    ## Open hours
    
    open_hrs = store.find_all('p')[2]
    openhr_list = open_hrs.get_text(separator = '\n', strip = True).split('\n')
    if len(openhr_list) > 7:
        store_details['Open Hours'].append('\n'.join(openhr_list[:-1]))
    else:
        store_details['Open Hours'].append('\n'.join(openhr_list))
    
   
    store_details['Store Name'] = store_info_dict['Store Name']
    


In [408]:
store_info_df = pd.DataFrame(store_info_dict)
store_info_df.head(5)

,Store Name,Latitude,Longitude,Store Address,Suburb/City,Postal Code,Country,Phone
0,Ashmore,-27.9785290000,153.3807070000,"ASHMORE CITY, SHOP 30 208 CURRUMBURRA ROAD",ASHMORE CITY,QLD 4214,AU,07 5597 0342
1,Browns Plains,-27.6654730000,153.0450660000,18 COMMERCE DRIVE Shop J08,BROWNS PLAINS,QLD 4118,AU,07 3086 9653
2,Bundaberg,-24.8897960000,152.3219570000,7 JOHANNA BOULEVARD SHOP 3,KENSINGTON,QLD 4670,AU,07 4151 6026
3,Burleigh,-28.1010520000,153.4375660000,197-207 REEDY CREEK ROAD,BURLEIGH HEADS,QLD 4220,AU,07 5520 5231
4,Chancellor Park,-26.7199030000,153.0471750000,30 CHANCELLOR VILLAGE BOULEVARD SHOP 1,SIPPY DOWNS,QLD 4556,AU,07 5456 2144


In [409]:
store_details_df = pd.DataFrame(store_details)
brand_store_info = store_info_df.merge(store_details_df)

In [410]:
store_details_df.head(5)

,Store Name,Open Hours
0,Ashmore,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
1,Browns Plains,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
2,Bundaberg,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
3,Burleigh,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
4,Chancellor Park,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...


In [411]:
brand_store_info.head(5)

,Store Name,Latitude,Longitude,Store Address,Suburb/City,Postal Code,Country,Phone,Open Hours
0,Ashmore,-27.9785290000,153.3807070000,"ASHMORE CITY, SHOP 30 208 CURRUMBURRA ROAD",ASHMORE CITY,QLD 4214,AU,07 5597 0342,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
1,Browns Plains,-27.6654730000,153.0450660000,18 COMMERCE DRIVE Shop J08,BROWNS PLAINS,QLD 4118,AU,07 3086 9653,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
2,Bundaberg,-24.8897960000,152.3219570000,7 JOHANNA BOULEVARD SHOP 3,KENSINGTON,QLD 4670,AU,07 4151 6026,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
3,Burleigh,-28.1010520000,153.4375660000,197-207 REEDY CREEK ROAD,BURLEIGH HEADS,QLD 4220,AU,07 5520 5231,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...
4,Chancellor Park,-26.7199030000,153.0471750000,30 CHANCELLOR VILLAGE BOULEVARD SHOP 1,SIPPY DOWNS,QLD 4556,AU,07 5456 2144,MONDAY 8:30 AM to 5:30 PM\nTUESDAY 8:30 AM to ...


In [412]:
brand_store_info.to_csv('brand_store_info.csv', encoding = 'utf-8', index = False)

## Bonus

In [321]:
from selenium import webdriver 
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
 
url = 


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Pass the defined options objects to initialize the web driver 
driver = webdriver.Chrome(options=options)
# Set an implicit wait of 5 seconds to allow time for elements to appear before throwing an exception
driver.implicitly_wait(5)
driver.get(url) 
time.sleep(20)

In [209]:
page_html = BeautifulSoup(page_source, 'html.parser')

In [226]:
appn_soup = BeautifulSoup(page_html, "html.parser")

In [322]:
html = driver.page_source

In [323]:
df = pd.read_html(html)

In [308]:
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find('table', class_ = 'table.table-hover.desktop')

In [309]:
print(table)

None


In [214]:
page_html.find_all('div', class_ = 'fixed-table-loading table table-hover')

[]

In [223]:
appn_soup

<head>
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-371870-65&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-2L61S8D6Z3&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" id="infogram-async" src="https://e.infogram.com/js/dist/embed-loader-min.js"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-5XQCL7L"></script><script>
//@@ force plan.sa.gov.au if come to dit domain
-1<location.href.indexOf("www.dit.sa.gov.au/saplanningportal")?window.location=window.location.href.replace("www.dit.sa.gov.au/saplanningportal","plan.sa.gov.au"):-1<location.href.indexOf("dit.sa.gov.au/saplanningportal")?window.location=window.location.href.replace("di

In [270]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<script>
//@@ force plan.sa.gov.au if come to dit domain
-1<location.href.indexOf("www.dit.sa.gov.au/saplanningportal")?window.location=window.location.href.replace("www.dit.sa.gov.au/saplanningportal","plan.sa.gov.au"):-1<location.href.indexOf("dit.sa.gov.au/saplanningportal")?window.location=window.location.href.replace("dit.sa.gov.au/saplanningportal","plan.sa.gov.au"):0==window.location.hostname.indexOf("www")&&(window.location=window.location.href.replace("//www.plan","//plan"));"http:"==location.protocol&&(location.href=location.href.replace("http://plan","https://plan"));
</script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer',

In [251]:
print(table)

None
